In [ ]:
# in case you are on google colab
%pip install qiskit pylatexenc sympy

In [ ]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, AncillaRegister, Parameter
from qiskit.quantum_info import Statevector, Operator
import pylatexenc

import matplotlib.pyplot as plt

import numpy as np

# Quantum Fourier Transform

$n = 1$ case:

In [ ]:
n = 1

quantum_register = QuantumRegister(size=n, name="x")

QFT_circuit = QuantumCircuit(quantum_register, name="QFT circuit")

QFT_circuit.h(quantum_register)

QFT_circuit.draw(output="mpl")

$n = 2$ case:

In [ ]:
n = 2

quantum_register = QuantumRegister(size=n, name="x")

QFT_circuit = QuantumCircuit(quantum_register, name="QFT circuit")

QFT_circuit.swap(quantum_register[0], quantum_register[1])

QFT_circuit.h(quantum_register[0])
QFT_circuit.cp(2 * np.pi / 2**2, quantum_register[0], quantum_register[1])
QFT_circuit.h(quantum_register[1])

QFT_circuit.draw(output="mpl")

In [ ]:
n = 3

quantum_register = QuantumRegister(size=n, name="x")

QFT_circuit = QuantumCircuit(quantum_register, name="QFT circuit")

### write the rest of the code below



###

QFT_circuit.draw(output="mpl")

## Below is an implementation using only Hadamard, $CX$, $CP \left( \tfrac{\pi}{2^a} \right)$, and SWAP gates.

(Can you rewrite to only use Hadamard, $R_Z \left( \tfrac{\pi}{2^a} \right)$, and $CX$ gates?)

In [ ]:
def quantum_fourier_transform(n):
    quantum_register = QuantumRegister(size=n, name="x")
    QFT_circuit = QuantumCircuit(quantum_register, name=f"QFT")

    for q, p in zip(quantum_register[:n >> 1], reversed(quantum_register[n >> 1:])):
        QFT_circuit.swap(q, p)

    for i, q in enumerate(quantum_register, start=1):
        QFT_circuit.h(q)
        for j, p in enumerate(quantum_register[i:], start=1):
            QFT_circuit.cp(np.pi / (1 << j), q, p)

    return QFT_circuit

def inverse_quantum_fourier_transform(n):
    quantum_register = QuantumRegister(size=n, name="x")
    inverse_QFT_circuit = QuantumCircuit(quantum_register, name=f"IQFT")

    for i, q in enumerate(reversed(quantum_register), start=1):
        for j, p in enumerate(reversed(quantum_register[n + 1 - i:]), start=1):
            inverse_QFT_circuit.cp(- np.pi / (1 << (i - j)), q, p)
        inverse_QFT_circuit.h(q)

    for q, p in zip(quantum_register[:n >> 1], reversed(quantum_register[n >> 1:])):
        inverse_QFT_circuit.swap(q, p)

    return inverse_QFT_circuit

In [ ]:
quantum_fourier_transform(1).draw(output="mpl")

In [ ]:
quantum_fourier_transform(2).draw(output="mpl")

In [ ]:
quantum_fourier_transform(3).draw(output="mpl")

In [ ]:
inverse_quantum_fourier_transform(3).draw(output="mpl")

# Draper's adder circuit

In [ ]:
def draper_adder(k, n):
    quantum_register = QuantumRegister(size=n, name="x")
    draper_adder_circuit = QuantumCircuit(quantum_register, name=f"{k} adder")
    
    draper_adder_circuit.compose(quantum_fourier_transform(n), inplace=True)

    draper_adder_circuit.barrier()
    
    # phaser part
    for idx, q in enumerate(reversed(quantum_register)):
        draper_adder_circuit.p(np.pi * k / (1 << idx), q)
    
    draper_adder_circuit.barrier()
    
    draper_adder_circuit.compose(inverse_quantum_fourier_transform(n), inplace=True)
    
    return draper_adder_circuit

In [ ]:
k, n = 2, 3
adder_circuit = draper_adder(k, n)

In [ ]:
adder_circuit.draw(output="mpl")

In [ ]:
O = Operator(adder_circuit).data
O

### Let's get rid of the noise to see that 'O' is indeed the matrix of shift-by-two.

In [ ]:
np.round(np.real(O)).astype(int)

# Fejér states:

For $k \in \mathbb{R}$: $| k \rangle_F := \tfrac{1}{2^n} \sum\limits_{x = 0}^{2^n - 1} \exp \left( \tfrac{2 \pi i}{2^n} \left( k - x \right) \right) | x \rangle = QFT^\dagger \circ P(k) \circ H^{\otimes n} | 0 \rangle_n$.

In [ ]:
n = 7
k = 2**(n - 1) + 0.5
n, 2**n, k

In [ ]:
quantum_register = QuantumRegister(size=n, name="x")

fejer_circuit = QuantumCircuit(quantum_register, name=r"circuit for | k \rangle_F")

### your code comes here



###

fejer_circuit.draw(output="mpl")

In [ ]:
fejer_distribution = Statevector(fejer_circuit).probabilities()

In [ ]:
plt.plot(np.arange(2**n), fejer_distribution)

### the two most likely configurations

In [ ]:
k_low = int(np.floor(k))
k_high = int(np.ceil(k))

k_low, k_high

In [ ]:
fejer_distribution[k_low] + fejer_distribution[k_high], 8 / np.pi**2

# Quantum Phase Estimation

Input:

$CU$: Controlled Unitary

$| \Psi \rangle$: eigenstate of $U$, that is, $U | \Psi \rangle = e^{i 2 \pi \theta} | \Psi \rangle$, with $\theta \in \left[ 0, 1 \right)$.

$n$: binary precision

Output:

An esitmate, $\theta^{(n)} \in \left[ 0, 1 \right)$, such that, $\left| \theta - \theta^{(n)} \right| < \tfrac{1}{2^{n + 1}}$, with high probability.

Note: More precisely the output is the $n$-qubit Fejér state $| 2^n \theta \rangle_F$.


# Example: random phase gate

In [ ]:
theta = np.random.rand()
theta

Let $U = P \left( \pi \theta \right) = \begin{pmatrix} 1 & 0 \\ 0 & e^{2 \pi i \theta} \end{pmatrix}$.

Now $| \Psi \rangle = | 1 \rangle$ is an eigenstate of $U$ with eigenvalue $e^{2 \pi i \theta}$.

Let us pick $n$ for precision.

In [ ]:
n = 5

# this computes the bits of the closest rational number with only n bits (why?)
theta_bits = (int(np.rint(2**n * theta)) >> np.arange(n))%2

theta, n, theta_bits

In [ ]:
quantum_register = QuantumRegister(size=n, name="x")
ancilla_register = AncillaRegister(size=1, name=r"\psi")

qpe_circuit = QuantumCircuit(quantum_register, ancilla_register, name="QPE")

### your code comes here




In [ ]:
qpe_circuit.draw(output="mpl")

In [ ]:
output_state = Statevector(qpe_circuit)
output_state.draw("latex")

### note: the first qubits is always $| 1 \rangle$, as it should

In [ ]:
output_distribution = (output_state.probabilities())[2**n:] # the first 2^n components can be ignored
output_distribution.sum() # check

In [ ]:
output_distribution

In [ ]:
plt.plot(np.arange(2**n), output_distribution)

In [ ]:
t = np.where(output_distribution >= 4 / np.pi**2)[0] # the most likely outcome
t

### the bits of the most likely outcome

In [ ]:
(t.reshape(-1,1) >> np.arange(n))%2

In [ ]:
theta_bits

### the corresponding guess

In [ ]:
theta_guess = t / 2**n
theta, theta_guess

In [ ]:
# precision
np.abs(theta_guess - theta), 1 / 2**(n+1)

In [ ]:
# expected theta
E_theta = np.dot(output_distribution, np.arange(2**n)) / 2**n
E_theta, theta

In [ ]:
#  deviation
sigma_theta = np.sqrt(np.dot(output_distribution, np.square(E_theta  - np.arange(2**n) / 2**n)))
sigma_theta